In [ ]:
# 导入库
import akshare as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# akshare 版本  
print(ak.__version__)

In [ ]:
# 获取白酒行业成分股数据

stocks = ak.stock_board_industry_cons_ths(symbol="白酒")
stocks.to_csv("白酒股票.csv")


In [ ]:
# 获取板块股票逐年利润表
profit_y_dict = dict()

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    if (code.startswith('00') != True):
        market ='sh'
    symbol = "{}{}".format(market, code)
    profit_y = ak.stock_profit_sheet_by_yearly_em(symbol=symbol)
    # profit_y.to_csv("profit_{}.csv".format(code))

    profit_y_dict[code] = profit_y

# print(profit_y_dict)

In [ ]:
# 获取板块股票归属母公司利润表

profit_dict = {}
for i in tqdm(range(len(stocks))):
    code = stocks.loc[i, "代码"]

    p = pd.Series(list(profit_y_dict[code]["PARENT_NETPROFIT"]), index=pd.to_datetime(profit_y_dict[code]["REPORT_DATE"]).dt.date)
    profit_dict[code] = p

df_parent_profit = pd.DataFrame(profit_dict)
# df_parent_profit.fillna(0, inplace=True)

In [ ]:
df_parent_profit['000858']

In [ ]:
df_parent_profit['000858'].to_csv("profit_000858.csv")

In [ ]:
df_eps = pd.DataFrame()
eps_dict = {}

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]
    
    s1 = pd.Series(list(profit_y_dict[code]["BASIC_EPS"]), index=pd.to_datetime(profit_y_dict[code]["REPORT_DATE"]))
    eps_dict["{}-{}".format(code, name)] = s1

df_eps = pd.DataFrame(eps_dict)
df_eps.fillna(0, inplace=True)

In [ ]:
df_eps.to_csv("eps.csv")

In [ ]:
df_eps.fillna(0, inplace=True)


In [ ]:
print(type(df_eps.index))

In [ ]:
df_eps['2010':]

In [ ]:
df_close

In [ ]:
df_eps['2010':].plot(kind='bar')

In [ ]:
print(df_eps.mean().sort_values(ascending=False))

In [ ]:
df_eps.sort_index(ascending=True).plot(kind="bar", colormap="cubehelix",legend=False)

In [ ]:
### 导出每股收益到csv
profiet_y = ak.stock_profit_sheet_by_yearly_em(symbol="sh603919")
print(profit_y["BASIC_EPS"][0:5])
profiet_y.to_csv("profit_603919.csv")

In [ ]:
df_stocks.sort_index(ascending=True).pct_change().plot(kind="bar", colormap="cubehelix")

In [ ]:
import datetime

trade_date_list= ak.tool_trade_date_hist_sina()
all_trade_date = pd.to_datetime(trade_date_list["trade_date"])
all_trade_date.index = all_trade_date
print(all_trade_date)

ts = pd.Series(np.random.randn(len(trade_date_index)), index=trade_date_index)

enddate = datetime.datetime.now().strftime("%Y%m%d")
print(ts)
print(ts['2016-01-01':enddate])

In [ ]:
# 导出白酒成分股到csv
stocks.to_csv("bj.csv")

In [ ]:
#股票走势
import datetime

startdate = datetime.date(2000,1,1)
enddate = datetime.datetime.now().strftime("%Y%m%d")

trade_date_list= ak.tool_trade_date_hist_sina()
all_trade_date = pd.to_datetime(trade_date_list["trade_date"])
all_trade_date.index = all_trade_date
trade_date_index = all_trade_date[startdate:enddate]

# df_close = pd.DataFrame()
cl_dict = {}

for i in tqdm(range(len(stocks))):
# for i in range(3):
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]
    stock_hist = ak.stock_zh_a_hist(symbol=code, period="daily", start_date=startdate.strftime("%Y%m%d"), end_date=enddate, adjust="hfq")
    
    stock_hist.to_csv(f"{code}-stock.csv")

    cl = stock_hist["收盘"]
    # if (len(cl) != len(trade_date_index)):
    #     print("股票数据长度不匹配:{}:{}-{}".format(len(cl), len(trade_date_index), code))
    #     continue

    ts = pd.Series(list(cl), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    ts_high = pd.Series(list(stock_hist["最高"]), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    ts_low = pd.Series(list(stock_hist["最低"]), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    # print(ts)
    # print(stock_hist["收盘"])
    cl_dict["{}".format(code)] = ts
df_close = pd.DataFrame(cl_dict)
  
# df_close.plot(subplots=True, figsize=(16,16))
# print(stock_hist)

In [ ]:
stocks

In [ ]:
ret = (1 + df_close.pct_change()).cumprod()
print(ret)
# ret.to_csv("ret.csv")
# df_close.to_csv("cl.csv")

# print(df_close.loc["2016-01-29",  :])
# print(df_close.resample('M').asfreq())

In [ ]:
ret.plot(kind="line")

In [ ]:
print(df_close['600519'].resample('M').last())


In [ ]:
print(df_close)

In [ ]:
### 月收益率
month_ret = df_close.pct_change().resample('M').apply(lambda x : ((1+x).prod() - 1))
print(month_ret)

In [ ]:
### 年度收益率
year_ret = df_close.pct_change().resample('Y').apply(lambda x : ((1+x).prod() - 1))
print(year_ret)
# year_ret.to_csv('year_ret.csv')

In [ ]:
### 作图区

ax1 = df_parent_profit['000858'].dropna().sort_index().plot(kind="bar")

pc = df_parent_profit['000858'].dropna().sort_index().pct_change()
pc.fillna(0, inplace=True)

ax2 = ax1.twinx()
ax2.set_ylabel('change rate')
ax2.set_ylim(-1,1)
pc.plot(kind='line', marker='*', color='red', ms=10)

In [ ]:
year_ret['000858'].plot(kind='bar')

In [ ]:
df_dividents = pd.DataFrame()
dividents_dict = {}

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]

    dividents = ak.stock_dividents_cninfo(symbol=code)
    # sg = pd.Series(list(dividents["送股比例"]), index=pd.to_datetime(dividents["日期"]))
    # zz = pd.Series(list(dividents["转增比例"]), index=pd.to_datetime(dividents["日期"]))
    dividents.to_csv(f"{code}-div.csv")
    px = pd.Series(list(dividents["派息比例"]), index=pd.to_datetime(dividents["报告时间"]))

    dividents_dict["{}-{}".format(code, name)] = px

df_dividents = pd.DataFrame(dividents_dict)
df_dividents.fillna(0, inplace=True)

In [215]:
import akshare as ak
from tqdm import tqdm 
import pandas as pd


def balance_sheet_analyze(industry : str, name: str):
    stocks = ak.stock_board_industry_cons_ths(symbol=industry)
    index = ['资产总计', '负债合计', '所有者权益(或股东权益)合计', '未分配利润']
    columns =  []
    # date_index = None
    report_all = pd.DataFrame()

    for i in range(len(stocks)):
        code = stocks.loc[i, "代码"]
        columns.append(code)

    for i in tqdm(range(len(stocks))):
        code = stocks.loc[i, "代码"]

        balance = ak.stock_financial_report_sina(stock=code, symbol=name)
        # if date_index is None:
            # print(pd.to_datetime(balance["报表日期"]))
        date_index = pd.to_datetime(balance["报表日期"])

        # if report_all is None:
        #     report_all = pd.DataFrame(
        #         # np.random.randn(204,1),
        #         columns=columns,
        #         index=pd.MultiIndex.from_product(
        #             [date_index, index]
        #         )
        #     )            
        report = pd.DataFrame(
            # np.random.randn(204,1),
            columns=[code],
            index=pd.MultiIndex.from_product(
                [date_index, index]
            )
        )   #             

        for idx in range(len(index)):
            # tmpDf = pd.DataFrame(balance[index[idx]], index=pd.to_datetime(balance["报表日期"]))
            report.loc[(slice(None), index[idx]), :] = balance[index[idx]].values
        # print(report_all)
        report_all = pd.concat([report_all, report], axis=1)
        # print(report_all)

    return report_all

In [217]:
df = balance_sheet_analyze("白酒", "资产负债表")

df.to_csv("bj_bal.csv", encoding="utf_8_sig")
# print(df)

100%|██████████| 20/20 [00:04<00:00,  4.43it/s]


In [248]:
df.loc['2022-09', '000568']

报表日期                      
2022-09-30  资产总计              43472427404.76
            负债合计              11496520216.58
            所有者权益(或股东权益)合计    31975907188.18
            未分配利润             24631260469.26
Name: 000568, dtype: object

In [267]:
df.sort_index(level=0).loc['2018-01':'2022-12', '资产总计', :].fillna(0).applymap(lambda x:format(float(x),','))

,,603198,600199,000568,000799,603369,600559,002304,600702,000995,600779,600809,000860,600696,000858,000596,002646,600197,600519,603919,603589
报表日期,,,,,,,,,,,,,,,,,,,,,
2018-03-31,资产总计,"5,666,227,873.03","3,065,424,585.22","20,147,440,936.4","2,564,719,390.68","7,229,899,420.0","3,287,571,523.17","44,363,922,274.08","4,733,560,899.41","241,395,919.47","2,991,083,821.58","10,455,642,162.38","17,931,712,937.82","745,548,425.0","79,647,898,938.15","11,503,386,290.94","2,794,059,027.45","3,204,000,306.09","134,404,941,948.3","2,377,604,741.4","7,218,685,752.28"
2018-06-30,资产总计,"5,160,158,558.19","2,988,409,338.35","20,469,045,840.59","2,557,646,569.07","7,216,617,036.03","5,325,147,342.42","41,304,386,431.57","4,577,045,656.92","237,563,017.8","3,099,626,711.68","10,545,702,832.06","18,316,879,786.0","731,555,225.98","80,430,592,528.77","11,210,703,795.7","2,746,230,310.0","2,999,762,442.47","129,914,832,498.21","2,523,612,590.7","7,646,121,693.11"
2018-09-30,资产总计,"5,491,549,118.32","3,001,491,151.9","19,758,590,902.28","2,578,329,285.1","7,740,189,279.1","5,748,623,155.35","45,178,466,366.55","4,495,791,427.44","235,136,124.01","2,963,667,893.41","10,264,553,240.26","17,805,226,062.78","690,448,720.8","76,123,846,574.96","12,498,719,843.68","2,719,659,416.47","2,991,407,963.52","147,147,672,376.16","2,747,404,508.77","7,587,803,826.99"
2018-12-31,资产总计,"6,421,363,351.49","3,194,700,354.64","22,604,929,596.42","2,870,051,531.68","8,587,648,943.19","5,478,659,222.09","49,563,767,816.22","4,842,358,854.79","228,498,240.72","3,197,979,886.33","11,828,963,704.04","19,850,831,298.43","414,410,304.16","86,094,265,733.09","12,509,928,449.72","2,939,556,538.56","3,329,234,715.7","159,846,674,736.01","2,700,487,304.8","8,842,255,205.51"
2019-03-31,资产总计,"6,113,691,648.67","3,215,128,003.6","23,926,523,365.13","2,851,674,731.07","8,339,887,403.02","5,894,457,316.59","49,691,158,148.41","4,985,314,213.03","218,727,891.36","3,436,506,910.15","12,969,295,580.57","20,402,982,363.95","530,958,808.65","93,909,828,419.76","13,659,931,600.41","2,758,259,855.3","4,186,958,832.11","159,888,176,447.28","2,771,327,331.02","8,691,813,986.38"
2019-06-30,资产总计,"5,584,424,597.17","3,704,514,194.24","25,009,088,799.06","2,854,869,439.58","8,372,486,064.35","5,396,682,830.97","45,285,902,343.2","5,139,834,925.7","228,974,722.77","3,859,137,624.93","13,547,757,108.26","20,172,871,786.78","406,111,482.06","88,082,807,409.78","12,520,063,624.92","2,743,400,599.96","3,978,085,505.29","165,121,648,977.73","3,032,248,327.07","7,936,686,368.93"
2019-09-30,资产总计,"5,933,525,111.18","3,695,679,000.41","26,686,143,194.36","2,949,696,024.44","8,693,764,992.11","5,577,927,275.74","46,296,930,035.35","5,489,057,309.25","251,346,653.46","3,578,101,564.92","14,180,291,229.34","20,409,300,618.2","489,597,135.96","92,757,067,719.93","13,265,837,811.82","2,763,401,166.99","4,088,827,351.35","162,550,409,174.43","3,147,429,992.26","8,257,850,301.26"
2019-12-31,资产总计,"6,968,837,746.92","3,643,891,060.02","28,919,969,078.32","3,228,583,367.04","10,062,294,425.83","6,273,298,252.43","53,455,037,840.98","5,776,505,460.06","454,588,336.49","3,945,981,465.69","16,067,772,126.06","22,021,558,785.01","452,455,831.28","106,396,972,333.66","13,871,297,363.16","2,807,039,883.12","4,409,826,423.07","183,042,372,042.5","3,178,409,582.0","9,501,326,320.59"
2020-03-31,资产总计,"6,495,231,459.73","3,592,019,782.69","29,755,363,958.77","3,169,686,957.29","9,581,043,146.02","6,067,048,382.75","52,096,174,090.83","5,682,639,467.7","411,318,388.16","3,919,005,889.11","14,953,428,655.03","20,393,787,397.97","424,384,390.29","105,451,563,991.29","15,392,796,907.36","2,682,563,483.82","4,194,916,500.78","181,624,733,882.27","3,021,072,181.17","8,829,135,435.24"


In [270]:
df.sort_index(level=0).loc['2018-01':'2022-12', '资产总计', :]

,,603198,600199,000568,000799,603369,600559,002304,600702,000995,600779,600809,000860,600696,000858,000596,002646,600197,600519,603919,603589
报表日期,,,,,,,,,,,,,,,,,,,,,
2018-03-31,资产总计,5666227873.03,3065424585.22,20147440936.40,2564719390.68,7229899420.00,3287571523.17,44363922274.08,4733560899.41,241395919.47,2991083821.58,10455642162.38,17931712937.82,745548425.00,79647898938.15,11503386290.94,2794059027.45,3204000306.09,134404941948.30,2377604741.40,7218685752.28
2018-06-30,资产总计,5160158558.19,2988409338.35,20469045840.59,2557646569.07,7216617036.03,5325147342.42,41304386431.57,4577045656.92,237563017.80,3099626711.68,10545702832.06,18316879786.00,731555225.98,80430592528.77,11210703795.70,2746230310.00,2999762442.47,129914832498.21,2523612590.70,7646121693.11
2018-09-30,资产总计,5491549118.32,3001491151.90,19758590902.28,2578329285.10,7740189279.10,5748623155.35,45178466366.55,4495791427.44,235136124.01,2963667893.41,10264553240.26,17805226062.78,690448720.80,76123846574.96,12498719843.68,2719659416.47,2991407963.52,147147672376.16,2747404508.77,7587803826.99
2018-12-31,资产总计,6421363351.49,3194700354.64,22604929596.42,2870051531.68,8587648943.19,5478659222.09,49563767816.22,4842358854.79,228498240.72,3197979886.33,11828963704.04,19850831298.43,414410304.16,86094265733.09,12509928449.72,2939556538.56,3329234715.70,159846674736.01,2700487304.80,8842255205.51
2019-03-31,资产总计,6113691648.67,3215128003.60,23926523365.13,2851674731.07,8339887403.02,5894457316.59,49691158148.41,4985314213.03,218727891.36,3436506910.15,12969295580.57,20402982363.95,530958808.65,93909828419.76,13659931600.41,2758259855.30,4186958832.11,159888176447.28,2771327331.02,8691813986.38
2019-06-30,资产总计,5584424597.17,3704514194.24,25009088799.06,2854869439.58,8372486064.35,5396682830.97,45285902343.20,5139834925.70,228974722.77,3859137624.93,13547757108.26,20172871786.78,406111482.06,88082807409.78,12520063624.92,2743400599.96,3978085505.29,165121648977.73,3032248327.07,7936686368.93
2019-09-30,资产总计,5933525111.18,3695679000.41,26686143194.36,2949696024.44,8693764992.11,5577927275.74,46296930035.35,5489057309.25,251346653.46,3578101564.92,14180291229.34,20409300618.20,489597135.96,92757067719.93,13265837811.82,2763401166.99,4088827351.35,162550409174.43,3147429992.26,8257850301.26
2019-12-31,资产总计,6968837746.92,3643891060.02,28919969078.32,3228583367.04,10062294425.83,6273298252.43,53455037840.98,5776505460.06,454588336.49,3945981465.69,16067772126.06,22021558785.01,452455831.28,106396972333.66,13871297363.16,2807039883.12,4409826423.07,183042372042.50,3178409582.00,9501326320.59
2020-03-31,资产总计,6495231459.73,3592019782.69,29755363958.77,3169686957.29,9581043146.02,6067048382.75,52096174090.83,5682639467.70,411318388.16,3919005889.11,14953428655.03,20393787397.97,424384390.29,105451563991.29,15392796907.36,2682563483.82,4194916500.78,181624733882.27,3021072181.17,8829135435.24


In [259]:
df.sort_index(level=0).loc['2018-01':'2022-12', '资产总计'].applymap(lambda x:format(int(x),','))

KeyError: '资产总计'

idx = pd.IndexSlice
df.sort_index(level=0).loc["2018-01":"2022-09", :].apply(lambda x: format(x, ','))